In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [1]:
from langchain import __version__
print(__version__)

0.0.288


In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

t2v_embeddings = HuggingFaceEmbeddings(
    model_name='GanymedeNil/text2vec-large-chinese',
    cache_folder="~/SageMaker/.cache"
)

No sentence-transformers model found with name ~/SageMaker/.cache/GanymedeNil_text2vec-large-chinese. Creating a new one with MEAN pooling.
2023-Sep-14 04:26:28.0997 16965:16965 ERROR  TDRV:tdrv_get_dev_info                       No neuron device available
INFO:torch_xla:Letting libtpu.so load fail during _XLAC import. libtpu.so will be loaded from `libtpu` Python package when the ComputationClient is created.
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


In [5]:
# %pip install -U FlagEmbedding

In [6]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [7]:
model_name = "BAAI/bge-large-zh"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    cache_folder="~/SageMaker/.cache",
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-large-zh


In [8]:
bge_embeddings.query_instruction

'为这个句子生成表示以用于检索相关文章：'

In [9]:
txt1 = "布洛芬作为临床上常见的非甾体类药物，用于退热、消炎、止痛，更是家庭小药箱的常备。甚至很多人无论哪里疼都吃布洛芬，那么布洛芬真是万能止痛药？吃多了有什么副作用？哪些情况不能使用布洛芬？今天我们就来好好聊聊吧！"

txt2 = "布洛芬可以缓解多种疼痛，包括头痛、关节痛、偏头痛、牙痛、肌肉痛、神经痛、痛经、术后疼痛、腰痛、运动后损伤性疼痛；劳损、腱鞘炎、滑囊炎引起的疼痛；类风湿关节炎、骨关节炎、其他非类风湿关节炎及急性痛风。注意布洛芬的禁忌人群：孕妇、哺乳期妇女、备孕女性禁用；消化性溃疡或胃肠道出血患者禁用；心脏疾病患者，增加心脏病发作和卒中风险；患有肝病、肾病的患者，可能会加重病情。布洛芬的功效几乎涵盖了日常生活中会遇到的各种疼痛，但请大家注意，遇到某些疼痛时，如果盲目服用止痛药可能会耽误病情。其中最危险的是心脏疾病引起的疼痛，如心绞痛，发作时可能会导致心源性牙痛、后背疼痛、肩膀疼痛、胸部疼痛。发生时应尽快就医，不可自行服用止痛药，耽误最佳治疗时间。"

txt2_2 = "布洛芬可以缓解多种疼痛，包括头痛、关节痛、偏头痛、牙痛、肌肉痛、神经痛、痛经、术后疼痛、腰痛、运动后损伤性疼痛；劳损、腱鞘炎、滑囊炎引起的疼痛；类风湿关节炎、骨关节炎、其他非类风湿关节炎及急性痛风。注意布洛芬的禁忌人群：孕妇、哺乳期妇女、备孕女性禁用；消化性溃疡或胃肠道出血患者禁用；心脏疾病患者，增加心脏病发作和卒中风险；患有肝病、肾病的患者，可能会加重病情。如果是心绞痛，应尽快就医。"

txt3 = "正常服用布洛芬后可能出现以下副作用：恶心、呕吐、胃烧灼感、轻度消化不良、头痛、头晕、耳鸣、视物模糊、精神紧张、嗜睡、下肢水肿或体重骤增等。如果过量或长期服用，可能引起严重的副作用，包括严重胃肠道反应(如出血、溃疡、穿孔，表现为呕血、黑便、腹痛、腹胀等)、严重心血管血栓事件(可表现为胸痛、气短、无力、言语含糊等)、严重皮肤反应(如剥脱性皮炎、史约综合征)。如用药期间出现以上症状，或在第一次服药时出现皮疹或其他过敏反应，请停药就诊。因为布洛芬最常见的副作用是胃刺激，所以建议饭后服用！"

txt4 = "布洛芬不与它们同服：阿司匹林、甲氨蝶呤、地高辛肝素、呋塞米、降压药、降糖药等药物，不建议与布洛芬同时使用。"

In [49]:
??bge_embeddings.embed_query

Signature: bge_embeddings.embed_query(text: str) -> List[float]
Source:   
    def embed_query(self, text: str) -> List[float]:
        """Compute query embeddings using a HuggingFace transformer model.

        Args:
            text: The text to embed.

        Returns:
            Embeddings for the text.
        """
        text = text.replace("\n", " ")
        embedding = self.client.encode(
            self.query_instruction + text, **self.encode_kwargs
        )
        return embedding.tolist()
File:      ~/SageMaker/custom-miniconda/miniconda/envs/mt_python3/lib/python3.10/site-packages/langchain/embeddings/huggingface.py
Type:      method

In [44]:
embedding = bge_embeddings.embed_query(txt2)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
len(embedding)

1024

In [46]:
embeddings_txt = bge_embeddings.client.encode(txt2)
embeddings_1 = bge_embeddings.client.encode("心绞痛可以服用布洛芬吗?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [47]:
embeddings_txt @ embeddings_txt.T

174.55795

In [42]:
sentences = ["样例数据", "样例数据"]
emb_1 = bge_embeddings.client.encode(sentences)
similarity = emb_1 @ emb_1.T
print(similarity)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[174.55801 166.67966 149.29376 148.19781]
 [166.67966 166.92181 148.33249 148.04071]
 [149.29376 148.33249 202.81183 195.49854]
 [148.19781 148.04071 195.49854 198.49742]]


In [ ]:
print(txt2)
print('----------------------')
print(txt2_2)

In [19]:
sentences = [txt2, txt2_2, "心绞痛可以服用布洛芬吗", "心绞痛可以服用布洛芬吗?"]
emb_1 = bge_embeddings.client.encode(sentences)
similarity = emb_1 @ emb_1.T
print(similarity)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[174.55801 166.67966 149.29376 148.19781]
 [166.67966 166.92181 148.33249 148.04071]
 [149.29376 148.33249 202.81183 195.49854]
 [148.19781 148.04071 195.49854 198.49742]]


In [3]:
# ??load_evaluator

In [10]:
from langchain.evaluation import load_evaluator

bge_evaluator = load_evaluator("embedding_distance", embeddings=bge_embeddings)

In [13]:
bge_evaluator.evaluate_strings(prediction="I shall go", reference="I will go")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'score': 0.04215867779681992}

In [17]:
bge_evaluator.evaluate_strings(prediction="I shall go", reference="I should go")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'score': 0.017590257632474793}

In [15]:
bge_evaluator.evaluate_strings(prediction=txt2, reference="心绞痛可以服用布洛芬吗?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'score': 0.2038510231363202}

In [16]:
bge_evaluator.evaluate_strings(prediction=txt2_2, reference="心绞痛可以服用布洛芬吗?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'score': 0.1867064237170366}

In [38]:
# emb_1[1].tolist()

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
cosine_similarity(emb_1)

array([[0.9999993 , 0.97646433, 0.7934593 , 0.7961487 ],
       [0.97646433, 0.9999995 , 0.8061818 , 0.8132936 ],
       [0.7934593 , 0.8061818 , 1.0000001 , 0.9743607 ],
       [0.7961487 , 0.8132936 , 0.9743607 , 1.0000002 ]], dtype=float32)

In [52]:
# bge_embeddings.client

In [55]:
instruction = "为这个句子生成表示以用于检索相关文章："
query1 = "心绞痛可以服用布洛芬吗?"
query2 = "心绞痛可以服用布洛芬吗"

In [57]:
instruction = "为这个句子生成表示以用于检索相关文章："
passages = [txt2, txt2_2]
queries = [query1, query2]

q_embeddings = bge_embeddings.client.encode([instruction+q for q in queries])
p_embeddings = bge_embeddings.client.encode(passages)

scores = q_embeddings @ p_embeddings.T
print(scores)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[155.3031  154.70758]
 [154.2406  151.95166]]


In [39]:
from langchain.docstore.document import Document
docs = []
for txt in [txt1, txt2, txt2_2, txt3, txt4]:
    docs.append(Document(page_content=txt))

In [59]:
from langchain.vectorstores import FAISS, Chroma

# vs_faiss = FAISS.from_documents(docs, bge_embeddings)
# vs_chroma = Chroma.from_documents(docs, bge_embeddings)

In [60]:
??FAISS.from_documents

Signature:
FAISS.from_documents(
    documents: 'List[Document]',
    embedding: 'Embeddings',
    **kwargs: 'Any',
) -> 'VST'
Source:   
    @classmethod
    def from_documents(
        cls: Type[VST],
        documents: List[Document],
        embedding: Embeddings,
        **kwargs: Any,
    ) -> VST:
        """Return VectorStore initialized from documents and embeddings."""
        texts = [d.page_content for d in documents]
        metadatas = [d.metadata for d in documents]
        return cls.from_texts(texts, embedding, metadatas=metadatas, **kwargs)
File:      ~/SageMaker/custom-miniconda/miniconda/envs/mt_python3/lib/python3.10/site-packages/langchain/vectorstores/base.py
Type:      method

In [61]:
??HuggingFaceBgeEmbeddings.embed_documents

Signature: HuggingFaceBgeEmbeddings.embed_documents(self, texts: List[str]) -> List[List[float]]
Source:   
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Compute doc embeddings using a HuggingFace transformer model.

        Args:
            texts: The list of texts to embed.

        Returns:
            List of embeddings, one for each text.
        """
        texts = [t.replace("\n", " ") for t in texts]
        embeddings = self.client.encode(texts, **self.encode_kwargs)
        return embeddings.tolist()
File:      ~/SageMaker/custom-miniconda/miniconda/envs/mt_python3/lib/python3.10/site-packages/langchain/embeddings/huggingface.py
Type:      function

In [ ]:
from langchain.docstore.document import Document

documents = []
for prod in [prod1, prod2, prod3, prod4, prod5, prod6]:
    prod = prod.replace('\n', ' ')
    new_doc = Document(
        page_content=prod, metadata={'type': 'Prod'}
    )
    documents.append(new_doc)